In [13]:
import pandas as pd
# from examples.tools import analyze_data
path_prefix='/home/yutingh/data/xunfei/data/'
train_data = pd.read_csv(path_prefix+'train.csv',
                         encoding="utf-8", low_memory=False)
valid_data = pd.read_csv(path_prefix+'valid.csv',
                         encoding="utf-8", low_memory=False)
test_data = pd.read_csv(path_prefix+'test.csv',
                        encoding="utf-8", low_memory=False)
data = train_data.append([valid_data,test_data])                        
print(len(data))
data = data.drop_duplicates(['user_id','item_id']) 
print(len(data))
data = data.dropna()
print(len(data))
data = data.reset_index()
data 

864722
862169
862169


,index,user_id,item_id,score
0,0,4497,1,0.0
1,1,4511,1,1.0
2,2,4455,1,1.0
3,3,4502,1,1.0
4,4,4526,1,0.0
...,...,...,...,...
862164,173914,2320,2507,0.0
862165,173915,2232,2507,1.0
862166,173916,2324,2507,1.0
862167,173917,2163,2507,1.0


In [14]:
data=data.loc[:380000,:]
print(len(data))
group = data.groupby(['user_id'])
data =pd.DataFrame()
for gp in group:
    if len(gp[1])>15:
        data = data.append(gp[1])
print(len(data))
group =data.groupby(['item_id'])
data =pd.DataFrame()
for gp in group:
    if len(gp[1])>30 and len(gp[1])<=500:
        data = data.append(gp[1])
print(len(data))



380001
357238
263524


In [15]:
data1 = data.reset_index()
data1

,level_0,index,user_id,item_id,score
0,40,40,4444,1,1.0
1,45,45,4446,1,1.0
2,44,44,4448,1,1.0
3,57,57,4449,1,1.0
4,47,47,4451,1,1.0
...,...,...,...,...,...
263519,379768,380221,4368,1506,0.0
263520,379827,380280,4370,1506,1.0
263521,379971,380424,4373,1506,1.0
263522,379993,380446,4374,1506,1.0


In [16]:
# print(max(data1['user_id']),max(data1['item_id']),len(data1))
item_set=set(data1['item_id'])
item = pd.read_csv('/home/yutingh/data/xunfei/item.csv',
                         encoding="utf-8", low_memory=False)
know_set=set()
print(len(item))
df=pd.DataFrame()
for i,row in item.iterrows():
    if row['item_id'] in item_set:
        df = df.append(row,ignore_index=True)
        know_set.update(eval(row['knowledge_code']))                         
print(len(df))
# print(max(know_set))

2253
1180


In [17]:
print(max(know_set))

497


In [18]:
# 压缩user_id,item_id
from tqdm import tqdm
user_dict={}
for i,data in enumerate(data1['user_id'].unique()):
    user_dict[data]=i+1
data1[u'user_id'] = data1[u'user_id'].apply(lambda x :user_dict[x])
item_dict={}
for i,data in enumerate(data1['item_id'].unique()):
    item_dict[data]=i+1
data1[u'item_id'] = data1[u'item_id'].apply(lambda x :item_dict[x])
# print(len(data1))
print(max(data1['user_id']),max(data1['item_id']),len(data1))


6820 1180 263524


In [19]:
df[u'item_id'] = df[u'item_id'].apply(lambda x :item_dict[x])
skill_dict={}
for i,data in enumerate(know_set):
    skill_dict[data]=i+1


In [20]:
def skill2know(str):
    tmp_arr = eval(str)
    tmp_arr1=[]
    for j in tmp_arr:
        tmp_arr1.append(skill_dict[j])
    return tmp_arr1
print(df.head())
df[u'knowledge_code'] = df[u'knowledge_code'].apply(lambda x :skill2know(x))
df.head()


   item_id   knowledge_code
0       55  [231, 232, 234]
1     1010  [279, 288, 289]
2      172       [339, 341]
3      420            [100]
4      483       [336, 337]


,item_id,knowledge_code
0,55,"[163, 164, 166]"
1,1010,"[206, 214, 215]"
2,172,"[258, 260]"
3,420,[68]
4,483,"[255, 256]"


In [21]:
from sklearn.utils import shuffle
data1=shuffle(data1,random_state=1)

In [22]:
df.to_csv('/home/yutingh/icd/data/xunfei/item.csv', index=None)
data1=pd.DataFrame(data1,columns=['user_id','item_id','score'])
data1.to_csv('/home/yutingh/icd/data/xunfei/log.csv', index=None)

In [23]:
print(len(user_dict.keys()))
print(len(item_dict.keys()))

6820
1180
